In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import datetime
import re

from shapely.geometry import Point, shape, mapping
from shapely.ops import unary_union


#from geoband.API import *
import folium
from folium import plugins
import json
import geopandas as gpd
from geopandas import gpd
from haversine import haversine

In [2]:
directory = '/Users/seungji/Library/Mobile Documents/com~apple~CloudDocs/STDA/Project/'

In [3]:
censor = pd.read_csv(directory + 'data/censor.csv')

In [7]:
censor.head()

,시리얼,date,방문자수,자치구,주소,위도,경도
0,"2,992.0000",2021-01-04,"6,594.7167",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848
1,"2,992.0000",2021-01-05,"6,290.3333",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848
2,"2,992.0000",2021-01-06,"6,222.3667",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848
3,"2,992.0000",2021-01-07,"5,919.0333",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848
4,"2,992.0000",2021-01-08,"5,982.3000",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848


In [5]:
location = pd.read_csv(directory + 'data/sdot_censor_location.csv')
# 경기도 과천시 데이터 제거 
location = location.iloc[2:].reset_index(drop=True)
# gpd 로 데이터 변환
censor_gpd = gpd.GeoDataFrame(
    location,
    crs={'init': 'epsg:4326'},
    geometry=[Point(xy) for xy in zip(location.위도, location.경도)])
# 서울시 법정경계 표시 
border = gpd.read_file(directory + 'data/LARD_ADM_SECT_SGG_서울/LARD_ADM_SECT_SGG_11.shp',encoding = 'cp949' )
border.head()

,ADM_SECT_C,SGG_NM,SGG_OID,COL_ADM_SE,GID,geometry
0,11740,강동구,337,11740,127,"POLYGON ((969713.132 1948846.288, 969692.451 1..."
1,11710,송파구,1969,11710,128,"POLYGON ((968640.372 1944982.492, 968640.957 1..."
2,11680,강남구,33,11680,129,"POLYGON ((958696.114 1948605.678, 959195.920 1..."
3,11650,서초구,33,11650,130,"POLYGON ((958117.753 1940073.855, 958118.398 1..."
4,11620,관악구,33,11620,131,"POLYGON ((949321.250 1944035.054, 949323.256 1..."


In [8]:
censor_gpd = censor_gpd[censor_gpd.사이트명.isin(censor.시리얼.unique())].reset_index(drop=True)

### 지하철 2021년 1월부터 10월 데이터 전처리 
- 데이터 다 가져와서 일별을 월별로 합친다. 
- 10달의 지하철별 승하차인원수 얻을 수 있음

In [9]:
month = ['01','02','03','04','05','06','07','08','09','10']

In [10]:
tmp = pd.read_csv(directory + 'data/subway/CARD_SUBWAY_MONTH_202101.csv')
tmp = tmp.reset_index()
tmp.columns = ['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자','na']
tmp 

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자,na
0,20210101,일산선,백석,2724,2825,20210104,NaN
1,20210101,일산선,마두,1638,1652,20210104,NaN
2,20210101,3호선,을지로3가,1024,1009,20210104,NaN
3,20210101,3호선,종로3가,1255,1632,20210104,NaN
4,20210101,3호선,안국,2934,3029,20210104,NaN
...,...,...,...,...,...,...,...
18532,20210131,중앙선,팔당,1039,1089,20210203,NaN
18533,20210131,중앙선,운길산,866,797,20210203,NaN
18534,20210131,중앙선,양수,1207,1122,20210203,NaN
18535,20210131,중앙선,신원,249,160,20210203,NaN


In [11]:
for m in month:
    exec(f"temp = pd.read_csv(directory + 'data/subway/CARD_SUBWAY_MONTH_2021'+'{m}'+'.csv')")
    temp = temp.reset_index()
    temp.columns = ['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자','na']
    exec(f"subway{m} = temp.drop(columns = 'na')")

In [12]:
subway_all = pd.DataFrame()
for m in month:
    exec(f"subway_all = pd.concat([subway_all, subway{m}],axis = 0)")

In [13]:
# subway_all = pd.DataFrame()
# for i,m in enumerate(month): 
#     exec(f"tmp = subway{m}.groupby(['노선명','역명']).sum().reset_index()")
#     tmp['month'] = i+1
#     subway_all = pd.concat([subway_all, tmp],axis = 0)

In [14]:
subway_all['역명'] = subway_all['역명'].apply(lambda x : re.sub(r'\([^)]*\)','',x)) #괄호 포함 글자 제거
subway_all['역명'] = list(map(lambda x: x+'역', subway_all['역명']))

In [15]:
subway_all.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
0,20210101,일산선,백석역,2724,2825,20210104
1,20210101,일산선,마두역,1638,1652,20210104
2,20210101,3호선,을지로3가역,1024,1009,20210104
3,20210101,3호선,종로3가역,1255,1632,20210104
4,20210101,3호선,안국역,2934,3029,20210104


In [16]:
np.corrcoef(subway_all.승차총승객수, subway_all.하차총승객수)

array([[1.        , 0.99499759],
       [0.99499759, 1.        ]])

In [17]:
np.corrcoef(subway_all.승차총승객수, subway_all.하차총승객수-subway_all.승차총승객수)

array([[1.        , 0.03557281],
       [0.03557281, 1.        ]])

## 지하철 위치정보 가져오기 

In [20]:
subway_loc = pd.read_csv(directory + 'data/subway/subway_location.csv')
subway_loc.columns = ['subway_name','경도','위도']
subway_g = gpd.GeoDataFrame(subway_loc, geometry = gpd.points_from_xy(subway_loc.위도,subway_loc.경도),crs={'init': 'epsg:5181'})
subway_g.head()

,subway_name,경도,위도,geometry
0,용문역,127.5944,37.4826,POINT (37.483 127.594)
1,지평역,127.6297,37.4769,POINT (37.477 127.630)
2,수원시청역,127.0309,37.2619,POINT (37.262 127.031)
3,매탄권선역,127.0408,37.2525,POINT (37.253 127.041)
4,망포역,127.0569,37.2458,POINT (37.246 127.057)


## censor별로 근접한 지하철 뽑아내기 
- censor별로 500m 이내의 지하철을 뽑고 그 지하철의 승차 승객수와 하차 승객수의 평균, 승차-하차수 평균을 구하고자 한다. 
- 500m 이내에 지하철이 여러개 있는 경우 지하철들의 평균값으로 이용
- 500m 이내에 있지 않는 경우 아예 0으로 했을 경우 너무 차이가 많이 난다. 
    - 그러므로 해당 지하철에 대해서는 500-1000이내에 있을 경우에는 승하차수의 1/2
    - 1000초과일때는 승하차수의 1/4 
    로 처리했다. 
    - 가장 지하철역이 멀리 있는 경우는 1.94km 였다. 

In [21]:
censor_gpd['coord'] = list(zip(censor_gpd.위도,censor_gpd.경도))
subway_g['coord'] = list(zip(subway_g.위도,subway_g.경도))

In [22]:
subway_dic = dict()
for i, c in enumerate(censor_gpd.coord):
    temp_subwaynm = []
    for j, s in enumerate(subway_g.coord):
        if haversine(c, s) <= 0.50:
            temp_subwaynm.append(subway_g.subway_name[j])
    subway_dic[censor_gpd.사이트명[i]] = [temp_subwaynm,['<500']]

In [23]:
for c, sub in zip(subway_dic.keys(), subway_dic.values()):
    if sub[0] == []:
        temp = []
        c_coord = censor_gpd.coord[censor_gpd.사이트명 == c].values[0]
        for j, s in enumerate(subway_g.coord):
            temp.append(haversine(c_coord,s))
        if (min(temp)>=0.5)&(min(temp)<1.0):
            subway_dic[c][1] = ['500<1000']
        else: subway_dic[c][1] = ['1000<']
#         print(min(temp))
        subway_dic[c][0] = [subway_g.subway_name[np.argmin(temp)]]

In [24]:
subway_dic

{4019: [['건대입구역'], ['<500']],
 4040: [['안국역'], ['500<1000']],
 4043: [['명동역', '을지로입구역'], ['<500']],
 4005: [['압구정역'], ['500<1000']],
 4007: [['압구정로데오역'], ['<500']],
 4032: [['서울숲역'], ['<500']],
 4035: [['이태원역'], ['500<1000']],
 4001: [['신논현역', '강남역'], ['<500']],
 4002: [['압구정역'], ['<500']],
 4003: [['선릉역'], ['<500']],
 4004: [['선릉역'], ['<500']],
 4006: [['신사역'], ['<500']],
 4014: [['마곡역'], ['<500']],
 4015: [['마곡나루역'], ['<500']],
 4016: [['발산역'], ['<500']],
 4018: [['서울대입구역'], ['<500']],
 4021: [['녹천역'], ['<500']],
 4023: [['홍제역'], ['1000<']],
 4031: [['서울숲역', '뚝섬역'], ['<500']],
 4033: [['잠실역'], ['<500']],
 4034: [['목동역'], ['<500']],
 4036: [['구산역'], ['<500']],
 4037: [['역촌역'], ['<500']],
 4038: [['안국역'], ['500<1000']],
 4039: [['안국역'], ['500<1000']],
 4041: [['혜화역'], ['500<1000']],
 4042: [['창신역'], ['500<1000']],
 4045: [['시청역', '을지로입구역'], ['<500']],
 4046: [['동대문역사문화공원역', '동대문역'], ['<500']],
 4048: [['회현역'], ['<500']],
 4013: [['방화역'], ['<500']],
 4017: [['가양역'], ['<500']],
 4009: [[

In [25]:
m = folium.Map(
    location=[37.54238169591348,126.99068745821812],
 attr = '서울',
    zoom_start=12
)
    
for lat, lng, label in zip(location.위도,location.경도,location.자치구):
    folium.Marker(
      location = [lat, lng], 
      popup = label,
#       icon = folium.Icon(color='red', icon='info-sign')
     ).add_to(m) 

for lat, lng in zip(subway_loc.위도,subway_loc.경도):
    folium.Marker(
      location = [lat, lng], 
#       popup = label,
      icon = folium.Icon(color='red')
     ).add_to(m) 
border_geo = folium.GeoJson(data=border,
                       style_function=lambda x: {'fillColor': 'black', },).add_to(m)
folium.Popup(border.SGG_NM).add_to(m)    

folium.plugins.Fullscreen(position='topright',  
                   title='FULL SCREEN ON', 
                   title_cancel='FULL SCREEN OFF',
                   force_separate_button=True
                  ).add_to(m)

m

## censor 데이터 가져와서 결합하기 

In [28]:
subway_mrg = pd.DataFrame()
for c in censor.시리얼.unique():
    tmp = subway_all.loc[subway_all.역명.isin(subway_dic[c][0]),['사용일자','역명','승차총승객수','하차총승객수']].groupby(['사용일자']).mean().reset_index()
    tmp['시리얼'] = c
    if subway_dic[c][1]== ['<500']:
        tmp['총승객수'] = (tmp.승차총승객수 + tmp.하차총승객수)/2
    elif subway_dic[c][1]== ['500<1000']:   
        tmp['총승객수'] = (tmp.승차총승객수 + tmp.하차총승객수)/4
    else:
        tmp['총승객수'] = (tmp.승차총승객수 + tmp.하차총승객수)/8
    subway_mrg = pd.concat([subway_mrg,tmp],axis = 0)
    

In [29]:
censor.head()

,시리얼,date,방문자수,자치구,주소,위도,경도
0,"2,992.0000",2021-01-04,"6,594.7167",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848
1,"2,992.0000",2021-01-05,"6,290.3333",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848
2,"2,992.0000",2021-01-06,"6,222.3667",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848
3,"2,992.0000",2021-01-07,"5,919.0333",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848
4,"2,992.0000",2021-01-08,"5,982.3000",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848


In [30]:
subway_mrg.사용일자 = list(map(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8],subway_mrg.사용일자))

In [31]:
censor = pd.merge(censor, subway_mrg, left_on = ['시리얼','date'], right_on = ['시리얼','사용일자'], how = 'left')

In [32]:
censor = censor.drop(columns = ['사용일자','승차총승객수','하차총승객수'])

In [33]:
censor

,시리얼,date,방문자수,자치구,주소,위도,경도,총승객수
0,"2,992.0000",2021-01-04,"6,594.7167",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848,"15,223.5000"
1,"2,992.0000",2021-01-05,"6,290.3333",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848,"14,932.0000"
2,"2,992.0000",2021-01-06,"6,222.3667",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848,"15,729.0000"
3,"2,992.0000",2021-01-07,"5,919.0333",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848,"16,100.5000"
4,"2,992.0000",2021-01-08,"5,982.3000",구로구,서울특별시 구로구 구로동 733-28,37.4895,126.8848,"14,750.0000"
...,...,...,...,...,...,...,...,...
26314,"4,050.0000",2021-10-19,69.1000,서대문구,서울특별시 서대문구 현저동 101,37.5729,126.9556,"7,098.0000"
26315,"4,050.0000",2021-10-20,71.4667,서대문구,서울특별시 서대문구 현저동 101,37.5729,126.9556,"9,299.0000"
26316,"4,050.0000",2021-10-21,67.8667,서대문구,서울특별시 서대문구 현저동 101,37.5729,126.9556,"7,461.0000"
26317,"4,050.0000",2021-10-22,67.0000,서대문구,서울특별시 서대문구 현저동 101,37.5729,126.9556,"7,634.0000"


In [96]:
censor.to_csv(directory +'data/merge2.csv',index = False )